# Notebook 4.2 Speech Recognition

Speech recognition, also known as automatic speech recognition (ASR), is a technology that converts spoken words into written format or executes specific actions based on verbal commands. It involves machine learning models that analyze speech patterns, phonetics, and language structures to accurately transcribe and understand human speech.

[Whisper](https://openai.com/research/whisper), published by OpenAI, is a popular open-source model for bothe ASR and speech translation. This means that Whisper has the capability to transcribe speech in multiple languages and facilitate translation from those languages into English.

Due to its underlying Transformer-based encoder-decoder architecture, Whisper can be optimized effectively with BigDL-LLM INT4 optimizations. In this tutorial, we will guide you through building a speech recognition application on BigDL-LLM optimized Whisper model that can transcribe/translate audio files into text.

## 4.2.1 Install Packages

Follow instructions in [Chapter 2](../ch_2_Environment_Setup/) to setup your environment if you haven't done so. Then install bigdl-llm:

!pip install bigdl-llm[all]

Due to the requirement to process audio file, you will also need to install the `librosa` package for audio analysis.

In [ ]:
!pip install librosa

## 4.2.2 Download Audio Files

To begin, let's prepare some audio files. As an example, you can download audio files from the multilingual [common_voice](https://huggingface.co/datasets/common_voice/viewer/en/train) datasets. Here, one English audio file and one Chines audio file have been randomly selected. Feel free to choose different audio files according to your preferences.

In [ ]:
!wget -O audio_en.mp3 https://datasets-server.huggingface.co/assets/common_voice/--/en/train/5/audio/audio.mp3
!wget -O audio_zh.mp3 https://datasets-server.huggingface.co/assets/common_voice/--/zh-CN/train/0/audio/audio.mp3

Here you could play the downloaded audio:

In [ ]:
import IPython

IPython.display.display(IPython.display.Audio("audio_en.mp3"))
IPython.display.display(IPython.display.Audio("audio_zh.mp3"))

## 4.2.3 Load Pretrained Whisper Model

Now, let's load a pretrained Whisper model, e.g. [whisper-medium](https://huggingface.co/openai/whisper-medium) as an example. OpenAI has released pretrained Whisper models in various sizes, allowing you to choose the one that best fits your requirements. 

Simply use one-line `transformers`-style API in `bigdl-llm` to load `whisper-medium` with INT4 optimizations (by specifying `load_in_4bit=True`) as follows. Please note that model class `AutoModelForSpeechSeq2Seq` is used for Whisper:

In [ ]:
from bigdl.llm.transformers import AutoModelForSpeechSeq2Seq

model = AutoModelForSpeechSeq2Seq.from_pretrained(pretrained_model_name_or_path="openai/whisper-medium",
                                                  load_in_4bit=True)

## 4.2.4 Load Whisper Processor

A Whisper processor is also needed for both audio pre-processing, and post-processing model outputs from tokens to texts. Just use the official `transformers` API to load `WhisperProcessor`:

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(pretrained_model_name_or_path="openai/whisper-medium")

## 4.2.5 Transcribe English Audio

Once you have optimized the Whisper model using BigDL-LLM with INT4 optimization and loaded the Whisper processor, you are ready to begin transcribing the audio through model inference.

Let's start with the English audio file [audio_en.mp3](./audio_en.mp3). Before we feed it into Whisper processor, we need to extract sequence data from raw speech waveform:

In [ ]:
import librosa

data_en, sample_rate_en = librosa.load("audio_en.mp3", sr=16000)

> **Note**
>
> Since `WhisperFeatureExtractor` was trained using a sampling rate of 16000, it is important that the audio file is loaded here with the same sample rate to ensure accurate recognition.


We can then proceed to transcribe the audio file based on the sequence data, using exactly the same way as using official `transformers` API:

In [6]:
import torch
import time

# define task type
forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")

with torch.inference_mode():
    # extract input features for the Whisper model
    input_features = processor(data_en, sampling_rate=sample_rate_en, return_tensors="pt").input_features

    # predict token ids for transcription
    st = time.time()
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    end = time.time()

    # decode token ids into texts
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    print(f'Inference time: {end-st} s')
    print('-'*20, 'English Transcription', '-'*20)
    print(transcription)

Inference time: xxxx s
-------------------- English Transcription --------------------
[' Book me a reservation for mid-day at French Camp Academy.']


> **Note**
>
> `forced_decoder_ids` defines the context token for different language and task (transcribe or translate). If it is set to `None`, Whisper will automatically predict them.


## 4.2.6 Transcribe Chinese Audio and Translate to English

Then let's move to the Chinese audio [audio_zh.mp3](./audio_zh.mp3). Whisper can transcribe multilingual audio, and translate them into English. The only difference here is to define specific context token through `forced_decoder_ids`:

In [7]:
# extract sequence data
data_zh, sample_rate_zh = librosa.load("audio_zh.mp3", sr=16000)

# define Chinese transcribe task
forced_decoder_ids = processor.get_decoder_prompt_ids(language="chinese", task="transcribe")

with torch.inference_mode():
    input_features = processor(data_zh, sampling_rate=sample_rate_zh, return_tensors="pt").input_features
    st = time.time()
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    end = time.time()
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    print(f'Inference time: {end-st} s')
    print('-'*20, 'Chinese Transcription', '-'*20)
    print(transcription)

# define Chinese transcribe and translation task
forced_decoder_ids = processor.get_decoder_prompt_ids(language="chinese", task="translate")

with torch.inference_mode():
    input_features = processor(data_zh, sampling_rate=sample_rate_zh, return_tensors="pt").input_features
    st = time.time()
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    end = time.time()
    translation = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    print(f'Inference time: {end-st} s')
    print('-'*20, 'Chinese to English Translation', '-'*20)
    print(translation)

Inference time: xxxx s
-------------------- Chinese Transcription --------------------
['漢元鼎六年,武帝平定南越国,南越之地重新化郡,番禺仍为南海郡之。']
Inference time: xxxx s
-------------------- Chinese to English Translation --------------------
[' In the 6th year of Han Yuanding, the fifth emperor pacified the country of Nanyue, and the south of the country of Nanyue was re-']


## 4.2.7 What's Next?

In the upcoming chapter, we will explore the usage of BigDL-LLM in conjunction with langchain, a framework designed for developing applications with language models. With langchain integration, application development process could be simplified.